In [1]:
import json
import pyodbc
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from datetime import timedelta
from datetime import datetime, timezone

def divzero(a,b):
    if b != 0:
        result = a / b
    else:
        result = 1
    return result

import warnings
warnings.filterwarnings('ignore')

today = date.today()
now = datetime.now(timezone.utc)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [2]:
file = {
    "variable": {
        "variable1": "variable1",
        "variable2": 10
    }
}

In [3]:
#rules
variable1 = np.array(list({'variable1'}))
variable2 = 3

#cutoff internal score
cutoff = 100

In [4]:
#Internal rules

def rule_1():
    if file['variable']['variable1'] == variable1:
        result = {
            "rule_01": {
                "rulename": "variable1",
                "result" : "approved",
                 }
            }
    else:
        result = {
            "rule_01": {
                "rulename": "variable1",
                "result" : 'rejected',
                 }
            }
    return result

r1 = rule_1()

def rule_2():
    if (r1['rule_01']['result'] != 'approved'):
        result = {
            "rule_02": {
                "rulename": "",
                "result" : "",
                 }
            }          
    elif file['variable']['variable2'] >= variable2:
        result = {
            "rule_02": {
                "rulename": "variable2",
                "result" : "approved"                 }
            }
    else:
        result = {
            "rule_02": {
                "rulename": "variable2",
                "result" : 'rejected'
                 }
            }
    return result

r2 = rule_2()

#final decision output
def final_rules_result():
    if (r1['rule_01']['result'] == 'approved') & (r2['rule_02']['result'] == 'approved'):
        result = {'result': 'approved'}
    else:
        result = {'result': 'rejected'}
    return result

#all rule decisions
rules = dict(r1, **r2)

In [5]:
#Scorecards based on logistic model

def variable01():
    if final_rules_result()['result'] == 'approved':
        if file['variable']['variable1'] == 'clust':
            value = file['variable']['variable1']
            points = 100
            bucket = 1
            label = 'variable1'
            bucketlabel = "variable1"

        elif file['variable']['variable1'] == 'clust2':
            value = file['variable']['variable1']
            points = 50
            bucket = 2
            label = 'variable1'
            bucketlabel = "variable1"

        else:
            value = file['variable']['variable1']
            points = 10
            bucket = 7
            label = 'variable'
            bucketlabel = "variable"
    
    else:
        value = file['variable']['variable1']
        points = 0
        bucket = 0
        label = 'variable1'
        bucketlabel = "variable1"
            
    return {'value': value, 'points': points, 'label': label, 'bucket': bucket, 'bucketlabel': bucketlabel}

v1 = variable01()

def variable02():
    if final_rules_result()['result'] == 'approved':
        if 0 <= file['variable']['variable2'] < 32:
            value = file['variable']['variable2']
            points = 100
            bucket = 1
            label = 'variable2'
            bucketlabel = "variable2"

        elif 32 <= file['variable']['variable2'] < 1000:
            value = file['variable']['variable2']
            points = 50
            bucket = 2
            label = 'variable2'
            bucketlabel = "variable2"

        else:
            value = file['variable']['variable2']
            points = 10
            bucket = 10
            label = 'variable2'
            bucketlabel = "variable2"
            
    else:
        value = file['variable']['variable2']
        points = 0
        bucket = 0
        label = 'variable2'
        bucketlabel = "variable2"
        
    return {'value': value, 'points': points, 'label': label, 'bucket': bucket, 'bucketlabel': bucketlabel}

v2 = variable02()

def finalscore():
    score = v1['points'] + v2['points']
    return score

def scoredecision():
    if finalscore()>=cutoff:
        result = 'approved'
    else:
        result = 'rejected'
    return result

In [6]:
def finaldecision():
    if (final_rules_result()['result'] == 'approved') & (scoredecision()>='approved'):
        result = 'approved'
    else:
        result = 'rejected'
    return result

In [7]:
{'final_decision': finaldecision(),
 'rules_decision': final_rules_result()['result'],
 'score_decision': scoredecision(),
 'score': finalscore(),
 'rules': rules,
 'scorecard': {'variable_01': variable01()
               ,'variable_02': variable02()
              }
}

{'final_decision': 'approved',
 'rules_decision': 'approved',
 'score_decision': 'approved',
 'score': 110,
 'rules': {'rule_01': {'rulename': 'variable1', 'result': 'approved'},
  'rule_02': {'rulename': 'variable2', 'result': 'approved'}},
 'scorecard': {'variable_01': {'value': 'variable1',
   'points': 10,
   'label': 'variable',
   'bucket': 7,
   'bucketlabel': 'variable'},
  'variable_02': {'value': 10,
   'points': 100,
   'label': 'variable2',
   'bucket': 1,
   'bucketlabel': 'variable2'}}}